# Our Task: Find similar company names

Now that we have completed our Python Primer, let's try to complete a real task, while at the same time trying to practice loops, iterations, and other Python functionality that we studied.

### Matching Company Names

Quite often, in our data, we have entries represented as strings that refer to the same entity but have different string representations (e.g., McDonald's vs McDonalds vs McDonald‎). We want to write code that will help in the task of finding such similar entries in our data.

Our task can be broken in the following tasks:
* **Step 1**: Read the data from a file into a list of strings in memory (one string per line). For our source of company names, we can use either the list of restaurant names from the Restaurant Inspection dataset (see our prior session), and/or use a list of companies from a file available at https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/companies.txt. We need to write Python code that will read these files and return a list of company names.
* **Step 2**: We will then need to figure out how to compare two strings and find their similarity. For that, we need to write a function that takes as input two strings, and returns back their similarities (we will see below how to do that).
* **Step 3**: We will need to write a function that takes as input a company name, and returns back a list of matching company names, together with their similarity. Ideally, we would like to also sort the list based on the similarity (highest similarity metrics on top). As part of our learning process, we will use the _list comprehension_ approach to create the list. We will also use tuples as the elements of the list, so that the list contain elements such as `[("McDonalds", 0.88), ("McDonald's", 0.81),....]`.
* **Step 4**: In the final step, we will just perform the similarity computation across all companies in the dataset.


### STEP 0: Fetch the data

In [1]:
# This is a list of banks
!curl https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/companies.txt -o /home/ubuntu/data/bank-names.txt
    
# This is a list of NYC restaurants (also using this dataset for the regular expression notebooks)
!curl https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/uniquenames.txt -o /home/ubuntu/data/restaurant-names.txt   

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22747  100 22747    0     0  41852      0 --:--:-- --:--:-- --:--:-- 41891
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  364k  100  364k    0     0  1566k      0 --:--:-- --:--:-- --:--:-- 1569k


### STEP 1: Read the list of names from a file and create a list of names

In [2]:
# STEP 1: Read the list of names from a file and create a list of names


### STEP 2: Implement the similarity function

### Computing the similarity between two strings

There are many ways that we can calculate the similarity between two strings (e.g., see [1](http://en.wikipedia.org/wiki/String_metric), [2](http://en.wikipedia.org/wiki/Approximate_string_matching), [3](http://en.wikipedia.org/wiki/Plagiarism_detection), [4](https://www.cs.cmu.edu/~pradeepr/papers/ijcai03.pdf), [5](http://en.wikipedia.org/wiki/Category:String_similarity_measures), and links within). 

For our example, we will use the q-gram similarity metric. What is a q-gram? Simply a sequency of q-consecutive characters in the string. For example, the name "Panos" has the following 2-grams: "Pa", "an", "no", "os". (We can also add "#P" and "s#" if we want to capture the prefix and suffixes.) Strings that share a large number of q-grams are often similar. 

Hence, we can compute the similarity of two strings by first computing the set of q-grams for each string, and then compute the Jaccard coefficient between these sets. The [Jaccard coefficient](http://en.wikipedia.org/wiki/Jaccard_index) is simply defined as the size of the intersection divided by the size of the union of the two sets.

So, our task can be broken into two functions: 
* The first function takes a string and returns a list of the q-grams for the string, and 
* The second function takes as input two sets, and returns back their Jaccard coefficient.

In [10]:
# STEP 2: Now we implement our similarity function

# This returns a list of q-grams for a name
# getQgrams("Panos", 1) should return ["P", "a", "n", "s"]
# getQgrams("Panos", 2) should return ["Pa", "an", "no", "os"]
# etc
def getQgrams(name, q):
    pass
    


['##N', '#Ne', 'New', 'ew ', 'w Y', ' Yo', 'Yor', 'ork', 'rk ', 'k U', ' Un', 'Uni', 'niv', 'ive', 'ver', 'ers', 'rsi', 'sit', 'ity', 'ty#', 'y##']


In [11]:
# This function takes as input two names, computes their qgrams, and then computes
# the Jaccard coefficient (=intersection/union) of the two sets of qgrams
def computeSimilarity(name1, name2, q):
    qgrams1 = getQgrams(name1.upper(),q)
    qgrams2 = getQgrams(name2.upper(),q)
    common_qgrams = set(qgrams1) & set(qgrams2)
    all_qgrams = set(qgrams1) | set(qgrams2)
    #print "Qgrams1:", qgrams1
    #print "Qgrams2:", qgrams2
    #print "Common:", common_qgrams
    #print "All:", all_qgrams
    #print "Similarity:", 1.0 * len(common_qgrams) / len(all_qgrams)
    similarity = 1.0 * len(common_qgrams) / len(all_qgrams)
    return similarity
    
print computeSimilarity("Peter", "Pete", 2)

0.571428571429


### STEP 3: Compute similarity of a company against a list of company names

We now create a function that accepts a company name and a list of companies, and computes their similarity. This part will get us to exercise our for-loops, and also illustrate how we can use lists and tuples.

**Sorting a list of tuples**:_This part is a little bit advanced for now, so I will just give the code below. (Solution taken from http://stackoverflow.com/questions/3121979/how-to-sort-list-tuple-of-lists-tuples). Here is a small example below, which we will reuse in our function:_

In [12]:
data = [("Panos",0.5), ("Peter",0.6), ("Pan", 0.4)]
data.sort(key=lambda tupl: tupl[1], reverse=True)  # sorts in place, in descending order, based on the second element of the tuple
print data

[('Peter', 0.6), ('Panos', 0.5), ('Pan', 0.4)]


In [13]:
# STEP 3: We now create a function that accepts a company name
# and a list of companies, and computes their similarity
# We have a threshold parameter (by default set to be 0.7)
# that restricts the results to only string pairs with similarity
# above the threshold
def companySimilarity(companyName, companyList, q, threshold = 0.7):
    result = []
    for c in companyList:
        similarity = computeSimilarity(companyName.upper(), c.upper(), q)
        if similarity > threshold:
            result.append( (c, similarity) )
    result.sort(key=lambda tupl: tupl[1], reverse=True)
    return result      
      
r = companySimilarity("STARBUCKS", company_names, 4, 0.25)
print len(r)

34


### Step 4: Perform the similarity computation across all companies in the dataset.

In [14]:
# STEP 4: We are almost done. We now just go through all the companies in the list
# and we call the companySimilarity function that computes the similar company names
# for all the companies in the list. We store the results in a dictionary, with the 
# key being the company name, and the value being a "list of tuples" with the 
# similar company names and the corresponding similarity value.
q = 3
threshold = 0.25

f = open("/home/ubuntu/data/restaurant-matches.txt", "w")

for company in company_names:
    matches = companySimilarity(company, company_names, q, threshold)
    for m in matches[1:4]:
        line = company + "," + m[0] + "," + str(m[1]) + "\n"
        f.write(line)
    
f.close()

In [15]:
!head -10 /home/ubuntu/data/restaurant-matches.txt

#1 GARDEN CHINESE,SAGAR CHINESE,0.36
#1 GARDEN CHINESE,BEST GARDEN CHINESE RESTAURANT,0.351351351351
#1 GARDEN CHINESE,CHINA GARDEN CHINESE RESTAURANT,0.351351351351
#1 ME. NICK'S,UNCLE NICK'S,0.318181818182
#1 SABOR LATINO RESTAURANT,SABOR LATINO RESTAURANT,0.766666666667
#1 SABOR LATINO RESTAURANT,LATINO RESTAURANT,0.566666666667
#1 SABOR LATINO RESTAURANT,SABOR LATINO SEAFOOD RESTAURANT,0.564102564103
$1.25 PIZZA,LJ PIZZA,0.352941176471
$1.25 PIZZA,MJ PIZZA,0.352941176471
$1.25 PIZZA,MY PIZZA,0.352941176471
